# Importing Packages

In [1]:
from pyblock2.driver.core import DMRGDriver, SymmetryTypes, MPOAlgorithmTypes
import numpy as np
import math
import matplotlib.pyplot as plt
import time

# Functions for Constructing Matrices

In [2]:
def vector_to_hermitian(upper_triangle_vector):
    # Check if the input vector is real-valued or complex-valued
    is_real = np.isrealobj(upper_triangle_vector)
    
    # Determine the size of the matrix (n x n)
    n = int(np.sqrt(2 * len(upper_triangle_vector) + 0.25) - 0.5)
    
    # Initialize an empty matrix with the appropriate type
    dtype = float if is_real else complex
    A = np.zeros((n, n), dtype=dtype)
    
    # Fill in the upper triangle
    indices = np.triu_indices(n)
    A[indices] = upper_triangle_vector
    
    # Assign values to the lower triangle
    i_lower = np.tril_indices(n, -1)
    if is_real:
        A[i_lower] = A.T[i_lower]  # Copy upper triangle values to lower triangle
    else:
        A[i_lower] = np.conj(A.T[i_lower])  # Assign conjugate transpose to lower triangle
    
    return A

# Definitions

In [3]:
params = {}
params['L'], params['N'], params['nmax'] = 64, 64, 4
params['L1'], params['L2'], params['L3'] = 30, 34, 64 
params['t'], params['U1'], params['U2'], params['U3'] = 1.0, 0.5, 10.0, 0.5

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SAny | SymmetryTypes.CPX, n_threads=4)

driver.set_symmetry_groups("U1")
Q = driver.bw.SX

In [4]:
# [Part A] Set states and matrix representation of operators in local Hilbert space
site_basis, site_ops = [], []

# Definig Operators
bdag = np.diag(np.sqrt(np.arange(1, params['nmax'] + 1)), k=-1) # b+
b = bdag.T # b
n = np.diag(np.arange(0, params['nmax'] + 1), k=0) # particle number 
parity = np.diag([(-1.)**n for n in range(params['nmax']+1)]) # Parity no.

for k in range(params['L']):
    basis = [(Q(i), 1) for i in range(params['nmax'] + 1)] 
    ops = {
        "": np.identity(params['nmax'] + 1), # identity
        "C": bdag, # b+
        "D": b,  # b
        "N": n, # particle number
        "P": parity, # Parity no.
    }
    site_basis.append(basis)
    site_ops.append(ops)

# Setting Up Hamiltonian

In [5]:
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()


for i in range(params['L']-1):
    b.add_term("CD", [i, i+1], -params['t'])
    b.add_term("CD", [i+1, i], -params['t'])

# Region 1 
for i in range(0, params['L2'], 1):
    b.add_term("CCDD", [i, i, i, i], params['U1'] / 2)
    b.add_term("CD", [i, i], - params['U1'] / 2)

# Region 2
for i in range(params['L1'], params['L2'], 1):
    b.add_term("CCDD", [i, i, i, i], params['U2'] / 2)
    b.add_term("CD", [i, i], - params['U2'] / 2)

# Region 3
for i in range(params['L2'], params['L3'], 1):
    b.add_term("CCDD", [i, i, i, i], params['U3'] / 2)
    b.add_term("CD", [i, i], - params['U3'] / 2)

In [6]:
# [Part C] Perform DMRG

start_time = time.time()
#########################################

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)

mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=1, occs=[1] * params['L'])
energy = driver.dmrg(
    mpo, 
    mps, 
    n_sweeps=100, 
    bond_dims=[50] * 10 + [100] * 5 + [200] * 5 + [500] * 5 + [1000] * 5, 
    noises=[1e-3] * 3 + [1e-5] * 3 + [0],  
    thrds=[1e-10] * 20, 
    dav_max_iter=100, 
    tol=1e-10, 
    iprint=1)
print("Final bond dimension =", mps.info.get_max_bond_dimension())

#########################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")



Sweep =    0 | Direction =  forward | Bond dimension =   50 | Noise =  1.00e-03 | Dav threshold =  1.00e-10
Time elapsed =      0.660 | E =    -142.4658243039 | DW = 4.69478e-10

Sweep =    1 | Direction = backward | Bond dimension =   50 | Noise =  1.00e-03 | Dav threshold =  1.00e-10
Time elapsed =      3.549 | E =    -143.1289663283 | DE = -6.63e-01 | DW = 5.60738e-08

Sweep =    2 | Direction =  forward | Bond dimension =   50 | Noise =  1.00e-03 | Dav threshold =  1.00e-10
Time elapsed =      7.164 | E =    -143.3223759939 | DE = -1.93e-01 | DW = 2.84053e-07

Sweep =    3 | Direction = backward | Bond dimension =   50 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      9.340 | E =    -143.4114361974 | DE = -8.91e-02 | DW = 1.59549e-07

Sweep =    4 | Direction =  forward | Bond dimension =   50 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =     12.232 | E =    -143.4808962693 | DE = -6.95e-02 | DW = 2.77359e-07

Sweep =    5 | Direction = backward 

In [ ]:
# kets = [driver.split_mps(mps, ir, tag="KET-%d" % ir) for ir in range(mps.nroots)]

## <font color='blue'>Current Measurement</font>

### <font color='black'>Constructing MPO for $(-1)^{\hat{N}_{R_{1}}}$ where $\hat{N}_{R_{1}} = \sum_{i \in R_{1}} n_{i}$</font>

In [7]:
b_n = driver.expr_builder()
Parity_L1 = "P" * params['L1'] 
lst_index_L1 = list(range(params['L1']))
# b_n.add_term(Parity_L1, lst_index_L1, 1)
b_n.add_term('PPPPPPPPPPPPPPPPPPPPPPPPPPPPPP',[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29], 1)
N1_op = driver.get_mpo(b_n.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)

### <font color='black'>Applying $\ket{\psi^{\prime}} = (-1)^{\hat{N}_{R_{1}}}\ket{\psi}$</font>

In [1]:
bra = driver.get_random_mps(tag="BRA_N1_op", bond_dim=50, nroots=1, occs=[1] * params['L'])
mps2 = driver.multiply(bra, N1_op, mps, iprint = 1)

NameError: name 'driver' is not defined

### <font color='black'>Time Evolution of $\ket{\psi^{\prime}}$:: $\ket{\psi(t)} = e^{-i\cdot t \cdot H_{AJJ}}\ket{\psi^{\prime}}$</font>

In [9]:
mps3 = driver.td_dmrg(mpo = mpo, ket = mps2, delta_t=0.1 * 1j, target_t = 1 * 1j, final_mps_tag="mps3", bond_dims = [700], hermitian=True, iprint = 1)

Time Evolution  DELTA T = RE      0.00000000 + IM      0.10000000
Time Evolution TARGET T = RE      0.00000000 + IM      1.00000000
Time Evolution   NSTEPS =         10


AttributeError: 'complex' object has no attribute 'deep_copy'

In [ ]:
# mps3 = driver.td_dmrg(mpo, mps2, delta_t=0.1, target_t=1, final_mps_tag="KET_TimeEvolveHAJJ")

In [ ]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyscf import gto, scf, lo

BOHR = 0.52917721092
R = 1.8 * BOHR
N = 6

mol = gto.M(atom=[['H', (i * R, 0, 0)] for i in range(N)], basis="sto6g", symmetry="c1", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)

mf.mo_coeff = lo.orth.lowdin(mol.intor('cint1e_ovlp_sph'))
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf, ncore=0, ncas=None, g2e_symm=8)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2 | SymmetryTypes.CPX, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

bond_dims = [150] * 4 + [200] * 4
noises = [1e-4] * 4 + [1e-5] * 4 + [0]
thrds = [1e-10] * 8

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, integral_cutoff=1E-8, iprint=1)
ket = driver.get_random_mps(tag="KET", bond_dim=150, nroots=1)
energy = driver.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)
print('Ground state energy = %20.15f' % energy)

isite = 2
mpo.const_e -= energy
eta = 0.005

dmpo = driver.get_site_mpo(op='D', site_index=isite, iprint=0)
dket = driver.get_random_mps(tag="DKET", bond_dim=200, center=ket.center, left_vacuum=dmpo.left_vacuum)
driver.multiply(dket, dmpo, ket, n_sweeps=10, bond_dims=[200], thrds=[1E-10] * 10, iprint=1)

impo = driver.get_identity_mpo()
dbra = driver.copy_mps(dket, tag='DBRA')

dt = 0.2
t = 500.0
nstep = int(t / dt)
rtgf = np.zeros((nstep, ), dtype=complex)
rtgf[0] = driver.expectation(dket, impo, dket)
for it in range(nstep - 1):
    if it % (nstep // 100) == 0:
        print("it = %5d (%4.1f %%)" % (it, it * 100 / nstep))
    dbra = driver.td_dmrg(mpo, dbra, -dt * 1j, -dt * 1j, final_mps_tag='DBRA', hermitian=True, bond_dims=[200], iprint=0)
    rtgf[it + 1] = driver.expectation(dbra, impo, dket)

## <font color='blue'>Evolution over sweeps</font>

In [ ]:
bonddims, discardedweights, energies = driver.get_dmrg_results()
plt.plot(energies.flatten(), marker='o')
plt.xlabel('Sweep', fontsize=12)
plt.ylabel('Energy', fontsize=12)
plt.show()
plt.plot(bonddims, marker='o')
plt.xlabel('Sweep', fontsize=12)
plt.ylabel('Bond dimension', fontsize=12)
plt.show()
plt.plot(discardedweights, marker='o')
plt.xlabel('Sweep', fontsize=12)
plt.ylabel('Discarded weight', fontsize=12)
plt.yscale('log')
plt.show()

## <font color='blue'>Entanglement</font>

In [ ]:
SvN = driver.get_bipartite_entanglement()

plt.plot(np.arange(1,params['L']), SvN, marker='o')
plt.xlabel('Bond', fontsize=12)
plt.ylabel('Entanglement entropy', fontsize=12)
plt.show()

## <font color='blue'>Single-particle boson correlations</font>

### Calculate

In [ ]:
start_time = time.time()

bosoncorr_mat = driver.get_npdm(mps, npdm_expr='CD', fermionic_ops='', mask=[0, 1])[0]
navg = np.diag(bosoncorr_mat)
print("Total number =", sum(navg))

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

### Site occupations

In [ ]:
plt.plot(np.arange(params['L']), navg, marker='o')
plt.xlabel('Site', fontsize=12)
plt.ylabel('Occupation', fontsize=12)
plt.show()

### Correlation matrix

In [ ]:
plt.matshow(bosoncorr_mat, cmap='viridis', vmin=bosoncorr_mat.min(), vmax=bosoncorr_mat.max())
plt.colorbar()
plt.xlabel('Site j', fontsize=12)
plt.ylabel('Site i', fontsize=12)
plt.title(r'Boson correlation $\langle \hat{b}_{i}^{\dagger} \hat{b}_{j} \rangle$')
plt.show()

### Correlation vs distance

In [ ]:
plt.plot(np.arange(1,params['L']//2+1), bosoncorr_mat[params['L']//2-1,params['L']//2:], marker='o')
plt.xlabel('Separation $r$', fontsize=12)
plt.ylabel(r'Correlation $\langle \hat{b}_{L/2}^{\dagger} \hat{b}_{L/2+r} \rangle$', fontsize=12)
plt.yscale('log')
plt.xscale('linear')
plt.show()

## <font color='blue'>On-site number distribution</font>

### Calculate

In [ ]:
start_time = time.time()

ndist = np.array(driver.get_npdm(mps, npdm_expr=proj_keys, fermionic_ops='')).T
print("Deviation from unit filling =", np.linalg.norm([ndist[i].sum()-1 for i in range(params['L'])]))

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

### Site-averaged number distribution

In [ ]:
plt.plot(np.mean(ndist, axis=0), marker='o')
plt.xlabel('$n$', fontsize=12)
plt.ylabel('Site-avgd distribution $p(n)$', fontsize=12)
plt.yscale('linear')
plt.show()

### Number fluctuation

In [ ]:
nsqavg = np.array([np.dot(np.arange(params['nmax']+1)**2, ndist[i]) for i in range(params['L'])])
numfluc = np.sqrt(nsqavg - navg*navg)
plt.plot(np.arange(params['L']), numfluc, marker='o')
plt.xlabel('Site', fontsize=12)
plt.ylabel('Number fluctuation', fontsize=12)
plt.title(f"Avg = {np.mean(numfluc)}")
plt.show()

### Calculate

In [ ]:
start_time = time.time()

nnexp_mat = driver.get_npdm(mps, npdm_expr='CDCD', fermionic_ops='', mask=[0, 0, 1, 1])[0]
densitycorr_mat = nnexp_mat - np.outer(navg, navg)
print("Avg number fluctuation =", np.mean(np.sqrt(np.diag(densitycorr_mat))))

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

### Correlation matrix

In [ ]:
plt.matshow(densitycorr_mat, cmap='seismic', vmin=-np.abs(densitycorr_mat).max(), vmax=np.abs(densitycorr_mat).max())
plt.colorbar()
plt.xlabel('Site j', fontsize=12)
plt.ylabel('Site i', fontsize=12)
plt.title(r'Density correlation $\langle \hat{n}_{i}, \hat{n}_{j} \rangle$')
plt.show()

### Correlation vs distance

In [ ]:
plt.plot(np.arange(1,params['L']//2+1), -densitycorr_mat[params['L']//2-1,params['L']//2:], marker='o')
plt.xlabel('Separation $r$', fontsize=12)
plt.ylabel(r'Correlation $-\langle \hat{n}_{L/2}, \hat{n}_{L/2+r} \rangle$', fontsize=12)
plt.yscale('log')
plt.xscale('linear')
plt.show()